# 1 - Kinetic particles

Topics covered in this tutorial:

- basic functionalities of [Particles6D and Particles5D](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#base-modules) classes
- particle boundary conditions
- particle drawing on a disc
- time stepping using some [Particle Propagators](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#particle-propagators)
- instantiating [Cuboid](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Cuboid), [HollowCylinder](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.HollowCylinder) and [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Tokamak) mappings
- instantiation of [ProjectedMHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_projected_mhd.html#module-struphy.fields_background.mhd_equil.projected_equils) object
- plotting Tokamak coordinates in Struphy


## Particles in a box

Let $\Omega \subset \mathbb R^3$ be a box (cuboid). We search for trajectories $(\mathbf x_p, \mathbf v_p): [0,T] \to \Omega \times \mathbb R^3$, $p = 0, \ldots, N-1$ that satisfy

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= 0 \qquad && \mathbf v_p(0) = \mathbf v_{p0}\,.
 \end{align}
$$

In Struphy, the position coordinates are updated in logical space $[0, 1]^3 = F^{-1}(\Omega)$, for instance with the Propagator [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) which we shall use in what follows.

In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -5
r1 = 5.0
l2 = -7
r2 = 7.0
l3 = -1.0
r3 = 1.0
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.pic.particles import Particles6D

Np = 15
bc = ["reflect", "reflect", "periodic"]
loading_params = {"seed": None}

# instantiate Particle object
particles = Particles6D(Np=Np, bc=bc, domain=domain, loading_params=loading_params)

In [ ]:
particles.draw_markers()

In [ ]:
particles.positions

In [ ]:
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
particles.velocities

In [ ]:
from matplotlib import pyplot as plt

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

fig = plt.figure()
ax = fig.gca()

for i, pos in enumerate(pushed_pos):
    ax.scatter(pos[0], pos[1], c=colors[i % 4])
    ax.arrow(
        pos[0], pos[1], particles.velocities[i, 0], particles.velocities[i, 1], color=colors[i % 4], head_width=0.2
    )

ax.plot([l1, l1], [l2, r2], "k")
ax.plot([r1, r1], [l2, r2], "k")
ax.plot([l1, r1], [l2, l2], "k")
ax.plot([l1, r1], [r2, r2], "k")
ax.set_xlim(-6.5, 6.5)
ax.set_ylim(-9, 9)
ax.set_title("Initial conditions");

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=True)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles)

In [ ]:
import math

import numpy as np

# time stepping
Tend = 10.0
dt = 0.2
Nt = int(Tend / dt)

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
alpha = np.ones(Nt + 1, dtype=float)

pos[0] = pushed_pos

time = 0.0
n = 0
while time < (Tend - dt):
    time += dt
    n += 1

    # advance in time
    prop_eta(dt)

    # positions on the physical domain Omega
    pos[n] = domain(particles.positions).T

    # scaling for plotting
    alpha[n] = (Tend - time) / Tend

In [ ]:
for i in range(Np):
    ax.scatter(pos[:, i, 0], pos[:, i, 1], c=colors[i % 4], alpha=alpha)

ax.plot([l1, l1], [l2, r2], "k")
ax.plot([r1, r1], [l2, r2], "k")
ax.plot([l1, r1], [l2, l2], "k")
ax.plot([l1, r1], [r2, r2], "k")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_xlim(-6.5, 6.5)
ax.set_ylim(-9, 9)
ax.set_title(f"{math.ceil(Tend / dt)} time steps (full color at t=0)")
fig

## Particles in a cylinder

We use the same setup as before but change the domain $\Omega$ to a cylinder. We explore two options for drawing markers in posittion space:

- uniform in logical space $[0, 1]^3 = F^{-1}(\Omega)$
- uniform on the cylinder $\Omega$

In [ ]:
from struphy.geometry.domains import HollowCylinder

a1 = 0.0
a2 = 5.0
Lz = 1.0
domain = HollowCylinder(a1=a1, a2=a2, Lz=Lz)

In [ ]:
# instantiate Particle object
Np = 1000
bc = ["remove", "periodic", "periodic"]
loading_params = {"seed": None}

particles = Particles6D(Np=Np, bc=bc, loading_params=loading_params)

# instantiate another Particle object
name = "test_uni"
loading_params = {"seed": None, "spatial": "disc"}
particles_uni = Particles6D(Np=Np, bc=bc, loading_params=loading_params)

In [ ]:
particles.draw_markers()
particles_uni.draw_markers()

In [ ]:
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos_uni = domain(particles_uni.positions).T

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.scatter(pushed_pos[:, 0], pushed_pos[:, 1], s=2.0)
circle1 = plt.Circle((0, 0), a2, color="k", fill=False)
ax = plt.gca()
ax.add_patch(circle1)
ax.set_aspect("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Draw uniform in logical space")

plt.subplot(1, 2, 2)
plt.scatter(pushed_pos_uni[:, 0], pushed_pos_uni[:, 1], s=2.0)
circle2 = plt.Circle((0, 0), a2, color="k", fill=False)
ax = plt.gca()
ax.add_patch(circle2)
ax.set_aspect("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Draw uniform on disc");

In [ ]:
# instantiate Particle object
Np = 15
bc = ["reflect", "periodic", "periodic"]
loading_params = {"seed": None}

particles = Particles6D(Np=Np, bc=bc, domain=domain, loading_params=loading_params)

In [ ]:
particles.draw_markers()

In [ ]:
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
fig = plt.figure()
ax = fig.gca()

for n, pos in enumerate(pushed_pos):
    ax.scatter(pos[0], pos[1], c=colors[n % 4])
    ax.arrow(
        pos[0], pos[1], particles.velocities[n, 0], particles.velocities[n, 1], color=colors[n % 4], head_width=0.2
    )

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Initial conditions");

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles)

In [ ]:
# time stepping
Tend = 10.0
dt = 0.2
Nt = int(Tend / dt)

pos = np.zeros((Nt + 1, Np, 3), dtype=float)
alpha = np.ones(Nt + 1, dtype=float)

pos[0] = pushed_pos

time = 0.0
n = 0
while time < (Tend - dt):
    time += dt
    n += 1

    # advance in time
    prop_eta(dt)

    # positions on the physical domain Omega
    pos[n] = domain(particles.positions).T

    # scaling for plotting
    alpha[n] = (Tend - time) / Tend

In [ ]:
# make scatter plot for each particle in xy-plane
for i in range(Np):
    ax.scatter(pos[:, i, 0], pos[:, i, 1], c=colors[i % 4], alpha=alpha)

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"{math.ceil(Tend / dt)} time steps (full color at t=0)")
fig

## Particles in a cylinder with a magnetic field

Let $\Omega \subset \mathbb R^3$ be a cylinder as before. Now, we search for trajectories $(\mathbf x_p, \mathbf v_p): [0,T] \to \Omega \times \mathbb R^3$, $p = 0, \ldots, N-1$ that satisfy

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= \mathbf v_p \times \mathbf B_0(\mathbf x_p) \qquad && \mathbf v_p(0) = \mathbf v_{p0}\,,
 \end{align}
$$

where $\mathbf B_0$ is a given magnetic field from an [MHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils.html#mhd-equilibria). 
In addition to the Propagator [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) for the position update, we shall use [PushVxB](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVxB) for the velocity update.

In [ ]:
from struphy.geometry.domains import HollowCylinder

a1 = 0.0
a2 = 5.0
Lz = 1.0
domain = HollowCylinder(a1=a1, a2=a2, Lz=Lz)

In [ ]:
# instantiate Particle object
Np = 20
bc = ["remove", "periodic", "periodic"]
loading_params = {"seed": None}

particles = Particles6D(Np=Np, bc=bc, loading_params=loading_params)

In [ ]:
particles.draw_markers()

In [ ]:
# positions on the physical domain Omega
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
fig = plt.figure()
ax = fig.gca()

for n, pos in enumerate(pushed_pos):
    ax.scatter(pos[0], pos[1], c=colors[n % 4])
    ax.arrow(
        pos[0], pos[1], particles.velocities[n, 0], particles.velocities[n, 1], color=colors[n % 4], head_width=0.2
    )

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Initial conditions");

In [ ]:
from struphy.propagators.propagators_markers import PushVxB

# default parameters of Propagator
opts_vxB = PushVxB.options(default=True)
print(opts_vxB)

In [ ]:
from struphy.fields_background.equils import HomogenSlab

B0x = 0.0
B0y = 0.0
B0z = 1.0
equil = HomogenSlab(B0x=B0x, B0y=B0y, B0z=B0z)

In [ ]:
# set domain for Cartesian MHD equilibrium
equil.domain = domain

In [ ]:
from struphy.feec.psydac_derham import Derham
from struphy.fields_background.projected_equils import ProjectedMHDequilibrium

# instantiate Derham object
Nel = [16, 16, 32]
p = [1, 1, 3]
spl_kind = [False, True, True]
derham = Derham(Nel=Nel, p=p, spl_kind=spl_kind)

# instantiate a projected MHD equilibrium object
proj_equil = ProjectedMHDequilibrium(equil, derham)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain
PushVxB.domain = domain
PushVxB.derham = derham

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles)
prop_vxB = PushVxB(particles, b2=proj_equil.b2)

In [ ]:
# time stepping
Tend = 10.0 - 1e-6
dt = 0.2
Nt = int(Tend / dt)

pos = []
alpha = np.ones(Nt + 1, dtype=float)

marker_col = {}
for marker in particles.markers_wo_holes:
    m_id = int(marker[-1])
    marker_col[m_id] = colors[int(m_id) % 4]
ids_wo_holes = []

time = 0.0
n = 0
while time < (Tend - dt):
    time += dt
    n += 1

    # advance in time
    prop_vxB(dt / 2)
    prop_eta(dt)
    prop_vxB(dt / 2)

    # positions on the physical domain Omega (can change shape when particles are lost)
    pos += [domain(particles.positions).T]

    # id's of non-holes
    ids_wo_holes += [np.int64(particles.markers_wo_holes[:, -1])]

    # scaling for plotting
    alpha[n] = (Tend - time) / Tend

In [ ]:
# make scatter plot for each particle in xy-plane
for po, ids, alph in zip(pos, ids_wo_holes, alpha):
    cs = []
    for ii in ids:
        cs += [marker_col[ii]]
    ax.scatter(po[:, 0], po[:, 1], c=cs, alpha=alph)

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"{math.ceil(Tend / dt)} time steps (full color at t=0)")
fig

## Particles in a Tokamak equilibrium

We use the same Propagators from the previous example but load a more complicated [MHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils.html#mhd-equilibria), namely from an ASDEX-Upgrade equilibrium stored in an EQDSK file.

Let us instatiate an [EQDSKequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.equils.EQDSKequilibrium) with many of its default parameters, except for the density:

In [ ]:
from struphy.fields_background.equils import EQDSKequilibrium

n1 = 0.0
n2 = 0.0
na = 1.0
equil = EQDSKequilibrium(n1=n1, n2=n2, na=na)
equil.params

Since [EQDSKequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.equils.EQDSKequilibrium) is an [AxisymmMHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.base.AxisymmMHDequilibrium), which in turn is a [CartesianMHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.base.CartesianMHDequilibrium), we are free to choose any mapping for the simulation (e.g. a Cuboid for Cartesian coordinates). In order to be conforming to the boundary of the equilibrium, we shall choose the [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Tokamak) mapping:

In [ ]:
from struphy.geometry.domains import Tokamak

Nel = (28, 72)
p = (3, 3)
psi_power = 0.6
psi_shifts = (1e-6, 1.0)
domain = Tokamak(equilibrium=equil, Nel=Nel, p=p, psi_power=psi_power, psi_shifts=psi_shifts)

In [ ]:
equil.domain = domain

The [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Tokamak) domain is a [PoloidalSplineTorus](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_base.html#struphy.geometry.base.PoloidalSplineTorus), hence

$$
 \begin{align*}
 x &= R \cos(\phi)\,,
 \\
 y &= -R \sin(\phi)\,,
 \\
 z &= Z\,,
 \end{align*}
$$

between Cartesian $(x, y, z)$- and Tokamak $(R, Z, \phi)$-coordinates holds, where $(R, Z)$ spans a poloidal plane. Moreover, the Tokamak coordinates are related to general torus coordinates $(r, \theta, \phi)$ via a polar mapping in the poloidal plane:

$$
 \begin{align*}
 R &= R_0 + r \cos(\theta)\,,
 \\
 Z &= r \sin(\theta)\,,
 \\
 \phi &= \phi\,.
 \end{align*}
$$

The torus coordinates are related to Struphy logical coordinates $\boldsymbol \eta = (\eta_1, \eta_2, \eta_3) \in [0, 1]^3$ as 

$$
 \begin{align*}
 r &= a_1 + (a_2 - a_1) \eta_1\,,
 \\
 \theta &= 2\pi \eta_2\,,
 \\
 \phi &= 2\pi \eta_3\,,
 \end{align*}
$$

where $a_2 > a_1 \geq 0$ are boundaries in the radial $r$-direction.
This can be seen for instance in the [HollowTorus](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.HollowTorus) mapping (more complicated angle parametrizations $\theta(\eta_1, \eta_2)$ are also available, but not discussed here).

Let us plot the equilibrium magnetic field strength:

1. in the poloidal plane at $\phi = 0$
2. in the top view at $z = 0$.

In [ ]:
import numpy as np

# logical grid on the unit cube
e1 = np.linspace(0.0, 1.0, 101)
e2 = np.linspace(0.0, 1.0, 101)
e3 = np.linspace(0.0, 1.0, 101)

# move away from the singular point r = 0
e1[0] += 1e-5

In [ ]:
# logical coordinates of the poloidal plane at phi = 0
eta_poloidal = (e1, e2, 0.0)
# logical coordinates of the top view at theta = 0
eta_topview_1 = (e1, 0.0, e3)
# logical coordinates of the top view at theta = pi
eta_topview_2 = (e1, 0.5, e3)

In [ ]:
# Cartesian coordinates (squeezed)
x_pol, y_pol, z_pol = domain(*eta_poloidal, squeeze_out=True)
x_top1, y_top1, z_top1 = domain(*eta_topview_1, squeeze_out=True)
x_top2, y_top2, z_top2 = domain(*eta_topview_2, squeeze_out=True)

print(f"{x_pol.shape = }")
print(f"{x_top1.shape = }")
print(f"{x_top2.shape = }")

In [ ]:
# generate two axes
fig, axs = plt.subplots(2, 1, figsize=(8, 16))
ax = axs[0]
ax_top = axs[1]

# min/max of field strength
Bmax = np.max(equil.absB0(*eta_topview_2, squeeze_out=True))
Bmin = np.min(equil.absB0(*eta_topview_1, squeeze_out=True))
levels = np.linspace(Bmin, Bmax, 51)

# absolute magnetic field at phi = 0
im = ax.contourf(x_pol, z_pol, equil.absB0(*eta_poloidal, squeeze_out=True), levels=levels)

# absolute magnetic field at Z = 0
im_top = ax_top.contourf(x_top1, y_top1, equil.absB0(*eta_topview_1, squeeze_out=True), levels=levels)
ax_top.contourf(x_top2, y_top2, equil.absB0(*eta_topview_2, squeeze_out=True), levels=levels)

# last closed flux surface, poloidal
ax.plot(x_pol[-1], z_pol[-1], color="k")

# last closed flux surface, toroidal
ax_top.plot(x_top1[-1], y_top1[-1], color="k")
ax_top.plot(x_top2[-1], y_top2[-1], color="k")

# limiter, poloidal
ax.plot(equil.limiter_pts_R, equil.limiter_pts_Z, "tab:orange")
ax.axis("equal")
ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("abs(B) at $\phi=0$")
fig.colorbar(im)
# limiter, toroidal
limiter_Rmax = np.max(equil.limiter_pts_R)
limiter_Rmin = np.min(equil.limiter_pts_R)

thetas = 2 * np.pi * e2
limiter_x_max = limiter_Rmax * np.cos(thetas)
limiter_y_max = -limiter_Rmax * np.sin(thetas)
limiter_x_min = limiter_Rmin * np.cos(thetas)
limiter_y_min = -limiter_Rmin * np.sin(thetas)

ax_top.plot(limiter_x_max, limiter_y_max, "tab:orange")
ax_top.plot(limiter_x_min, limiter_y_min, "tab:orange")
ax_top.axis("equal")
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("abs(B) at $Z=0$")
fig.colorbar(im_top);

In [ ]:
# instantiate Particle object
Np = 4
bc = ["remove", "periodic", "periodic"]
bufsize = 2.0

initial = [
    [0.501, 0.001, 0.001, 0.0, 0.0450, -0.04],  # co-passing particle
    [0.511, 0.001, 0.001, 0.0, -0.0450, -0.04],  # counter passing particle
    [0.521, 0.001, 0.001, 0.0, 0.0105, -0.04],  # co-trapped particle
    [0.531, 0.001, 0.001, 0.0, -0.0155, -0.04],
]

loading_params = {"seed": 1608, "initial": initial}

particles = Particles6D(Np=Np, bc=bc, loading_params=loading_params, bufsize=bufsize)

In [ ]:
particles.draw_markers()

In [ ]:
# positions on the physical domain Omega (x, y, z)
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
# compute R-coordinate
pushed_r = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

In [ ]:
labels = ["co-passing", "counter passing", "co_trapped", "counter-trapped"]

for n, (r, pos) in enumerate(zip(pushed_r, pushed_pos)):
    # poloidal
    ax.scatter(r, pos[2], c=colors[n % 4], label=labels[n])
    ax.arrow(
        r, pos[2], particles.velocities[n, 0], particles.velocities[n, 2] * 10, color=colors[n % 4], head_width=0.05
    )
    # topview
    ax_top.scatter(pos[0], pos[1], c=colors[n % 4], label=labels[n])
    ax_top.arrow(
        pos[0],
        pos[1],
        particles.velocities[n, 0],
        particles.velocities[n, 1] * 10,
        color=colors[n % 4],
        head_width=0.05,
    )

ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("Initial conditions")
ax.legend()
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("Initial conditions")
ax_top.legend()
fig

In [ ]:
# instantiate Derham object
Nel = [32, 72, 1]
p = [3, 3, 1]
spl_kind = [False, True, True]
derham = Derham(Nel=Nel, p=p, spl_kind=spl_kind)

# instantiate a projected MHD equilibrium object
proj_equil = ProjectedMHDequilibrium(equil, derham)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain
PushVxB.domain = domain
PushVxB.derham = derham

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles)
prop_vxB = PushVxB(particles, b2=proj_equil.b2)

In [ ]:
# time stepping
Tend = 3000.0 - 1e-6
dt = 0.2
Nt = int(Tend / dt)

pos = np.zeros((Nt + 2, Np, 3), dtype=float)
r = np.zeros((Nt + 2, Np), dtype=float)

pos[0] = pushed_pos
r[0] = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

time = 0.0
n = 0
while time < Tend:
    time += dt
    n += 1

    # advance in time
    prop_vxB(dt / 2)
    prop_eta(dt)
    prop_vxB(dt / 2)

    # positions on the physical domain Omega
    pushed_pos = domain(particles.positions).T

    # compute R-ccordinate
    pos[n] = pushed_pos
    r[n] = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

In [ ]:
# make scatter plot for each particle
for i in range(pos.shape[1]):
    # poloidal
    ax.scatter(r[:, i], pos[:, i, 2], c=colors[i % 4], s=1)
    # top view
    ax_top.scatter(pos[:, i, 0], pos[:, i, 1], c=colors[i % 4], s=1)

ax.set_title(f"{math.ceil(Tend / dt)} time steps")
ax_top.set_title(f"{math.ceil(Tend / dt)} time steps")
fig

## Guiding-centers in a Tokamak equilibrium

We now use the Propagators []() and []() in ASDEX-Upgrade equilibrium from the previous example.

For this we need to instantiate the [Particles5D]() class.

In [ ]:
from struphy.pic.particles import Particles5D

# instantiate Particle object
Np = 4
bc = ["remove", "periodic", "periodic"]
bufsize = 2.0

initial = [
    [0.501, 0.001, 0.001, -1.935, 1.72],  # co-passing particle
    [0.501, 0.001, 0.001, 1.935, 1.72],  # couner-passing particle
    [0.501, 0.001, 0.001, -0.6665, 1.72],  # co-trapped particle
    [0.501, 0.001, 0.001, 0.4515, 1.72],
]  # counter-trapped particle

loading_params = {"seed": 1608, "initial": initial}

particles = Particles5D(proj_equil, Np=Np, bc=bc, loading_params=loading_params, bufsize=bufsize)

In [ ]:
particles.draw_markers()

In [ ]:
# positions on the physical domain Omega (x, y, z)
pushed_pos = domain(particles.positions).T
pushed_pos

In [ ]:
# compute R-coordinate
pushed_r = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

In [ ]:
particles.velocities

In [ ]:
# generate two axes
fig, axs = plt.subplots(2, 1, figsize=(8, 16))
ax = axs[0]
ax_top = axs[1]

# min/max of field strength
Bmax = np.max(equil.absB0(*eta_topview_2, squeeze_out=True))
Bmin = np.min(equil.absB0(*eta_topview_1, squeeze_out=True))
levels = np.linspace(Bmin, Bmax, 51)

# absolute magnetic field at phi = 0
im = ax.contourf(x_pol, z_pol, equil.absB0(*eta_poloidal, squeeze_out=True), levels=levels)

# absolute magnetic field at Z = 0
im_top = ax_top.contourf(x_top1, y_top1, equil.absB0(*eta_topview_1, squeeze_out=True), levels=levels)
ax_top.contourf(x_top2, y_top2, equil.absB0(*eta_topview_2, squeeze_out=True), levels=levels)

# last closed flux surface, poloidal
ax.plot(x_pol[-1], z_pol[-1], color="k")

# last closed flux surface, toroidal
ax_top.plot(x_top1[-1], y_top1[-1], color="k")
ax_top.plot(x_top2[-1], y_top2[-1], color="k")

# limiter, poloidal
ax.plot(equil.limiter_pts_R, equil.limiter_pts_Z, "tab:orange")
ax.axis("equal")
ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("abs(B) at $\phi=0$")
fig.colorbar(im)
# limiter, toroidal
limiter_Rmax = np.max(equil.limiter_pts_R)
limiter_Rmin = np.min(equil.limiter_pts_R)

thetas = 2 * np.pi * e2
limiter_x_max = limiter_Rmax * np.cos(thetas)
limiter_y_max = -limiter_Rmax * np.sin(thetas)
limiter_x_min = limiter_Rmin * np.cos(thetas)
limiter_y_min = -limiter_Rmin * np.sin(thetas)

ax_top.plot(limiter_x_max, limiter_y_max, "tab:orange")
ax_top.plot(limiter_x_min, limiter_y_min, "tab:orange")
ax_top.axis("equal")
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("abs(B) at $Z=0$")
fig.colorbar(im_top);

In [ ]:
labels = ["co-passing", "counter passing", "co_trapped", "counter-trapped"]

for n, (r, pos) in enumerate(zip(pushed_r, pushed_pos)):
    # poloidal
    ax.scatter(r, pos[2], c=colors[n % 4], label=labels[n])
    # topview
    ax_top.scatter(pos[0], pos[1], c=colors[n % 4], label=labels[n])
    ax_top.arrow(pos[0], pos[1], 0.0, particles.velocities[n, 0] / 5, color=colors[n % 4], head_width=0.05)

ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("Initial conditions")
ax.legend()
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("Initial conditions")
ax_top.legend()
fig

In [ ]:
from struphy.propagators.propagators_markers import PushGuidingCenterBxEstar, PushGuidingCenterParallel

# default parameters of Propagator
opts_BxE = PushGuidingCenterBxEstar.options(default=True)
print(opts_BxE)

opts_para = PushGuidingCenterParallel.options(default=True)
print(opts_para)

In [ ]:
# pass simulation parameters to Propagator class
PushGuidingCenterBxEstar.domain = domain
PushGuidingCenterParallel.domain = domain

PushGuidingCenterBxEstar.derham = derham
PushGuidingCenterParallel.derham = derham

PushGuidingCenterBxEstar.projected_equil = proj_equil
PushGuidingCenterParallel.projected_equil = proj_equil

In [ ]:
# natural constants
mH = 1.67262192369e-27  # proton mass (kg)
e = 1.602176634e-19  # elementary charge (C)
mu0 = 1.25663706212e-6  # magnetic constant (N/A^2)

# epsilon equation parameter
A = 1.0  # mass number in units of proton mass
Z = 1  # signed charge number in units of elementary charge
unit_x = 1.0  # length scale unit in m
unit_B = 1.0  # magnetic field unit in T
unit_n = 1e20  # number density unit in m^(-3)
unit_v = unit_B / np.sqrt(unit_n * A * mH * mu0)  # Alfvén velocity unit
unit_t = unit_x / unit_v  # time unit

# cyclotron frequency and epsilon parameter
om_c = Z * e * unit_B / (A * mH)
epsilon = 1.0 / (om_c * unit_t)

print(f"{unit_x = }")
print(f"{unit_B = }")
print(f"{unit_n = }")
print(f"{unit_v = }")
print(f"{unit_t = }")
print(f"{epsilon = }")

In [ ]:
# instantiate Propagator object
opts_BxE["algo"]["tol"] = 1e-5
opts_para["algo"]["tol"] = 1e-5
prop_BxE = PushGuidingCenterBxEstar(particles, epsilon=epsilon, algo=opts_BxE["algo"])
prop_para = PushGuidingCenterParallel(particles, epsilon=epsilon, algo=opts_para["algo"])

In [ ]:
# time stepping
Tend = 100.0 - 1e-6
dt = 0.1
Nt = int(Tend / dt)

pos = np.zeros((Nt + 2, Np, 3), dtype=float)
r = np.zeros((Nt + 2, Np), dtype=float)

pos[0] = pushed_pos
r[0] = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

time = 0.0
n = 0
while time < Tend:
    time += dt
    n += 1

    # advance in time
    prop_BxE(dt / 2)
    prop_para(dt)
    prop_BxE(dt / 2)

    # positions on the physical domain Omega
    pushed_pos = domain(particles.positions).T

    # compute R-coordinate
    pos[n] = pushed_pos
    r[n] = np.sqrt(pushed_pos[:, 0] ** 2 + pushed_pos[:, 1] ** 2)

In [ ]:
# make scatter plot for each particle in xy-plane
for i in range(pos.shape[1]):
    # poloidal
    ax.scatter(r[:, i], pos[:, i, 2], c=colors[i % 4], s=1)
    # top view
    ax_top.scatter(pos[:, i, 0], pos[:, i, 1], c=colors[i % 4], s=1)

ax.set_title(f"{math.ceil(Tend / dt)} time steps")
ax_top.set_title(f"{math.ceil(Tend / dt)} time steps")
fig